# PRIMARY SETUPS

In [ ]:
!cd deepforest && pip install --verbose -e .

In [ ]:
!pip install -U scikit-learn

In [ ]:
# coding: utf-8

'''
Created on 21 мая 2016 г.

@author: keen
'''

from sklearn import datasets
import pickle

from scipy.sparse import csr_matrix
from sklearn import preprocessing
from numpy import ndarray
from numpy import asarray
from numpy.random import randint as rint
import datetime
from random import randint
import numpy
from sklearn.datasets import fetch_openml
#from sklearn.svm import SVC
import CO2_tree as co2t
import CO2_forest as co2f
from sklearn.metrics import accuracy_score

In [ ]:
TRAIN_SIZE = 5000
TEST_SIZE = 1000
SHAKE = False
NJOBS = 20

# Load all data

In [ ]:
data = {}

## MNIST

In [ ]:
x, Y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = Y.astype(int) + 1

x_train = x[:60000,:]
x_validate = x[60000:,:]

Y_train = Y[:60000]#[:6000]
Y_validate = Y[60000:]#[:3000]

data['MNIST'] = [[x_train,Y_train],[x_validate,Y_validate],(28,28,1)]

## USPS

In [ ]:
x, Y = fetch_openml("USPS", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = Y.astype(int) + 1

x_train = x[:7291,:]
x_validate = x[7291:,:]

Y_train = Y[:7291]#[:6000]
Y_validate = Y[7291:]#[:3000]
data['USPS'] = [[x_train,Y_train],[x_validate,Y_validate],(16,16,1)]

## CIFAR-10

In [ ]:
x, Y = fetch_openml("CIFAR_10", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = Y.astype(int) + 1

x_train = x[:50000,:]
x_validate = x[50000:,:]

Y_train = Y[:50000]#[:6000]
Y_validate = Y[50000:]#[:3000]
data['CIFAR'] = [[x_train,Y_train],[x_validate,Y_validate],(32,32,3)]

## LETTER

In [ ]:
x, Y = fetch_openml("letter", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = preprocessing.LabelEncoder().fit_transform(Y).astype(int) + 1

x_train = x[:15000,:]
x_validate = x[15000:,:]

Y_train = Y[:15000]#[:6000]
Y_validate = Y[15000:]#[:3000]
data['letter'] = [[x_train,Y_train],[x_validate,Y_validate],(4,4,1)]

## SATIMAGE

In [ ]:
x, Y = fetch_openml("satimage", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y =  preprocessing.LabelEncoder().fit_transform(Y).astype(int) + 1

x_train = x[:5000,:]
x_validate = x[5000:,:]

Y_train = Y[:5000]#[:6000]
Y_validate = Y[5000:]#[:3000]
data['satimage'] = [[x_train,Y_train],[x_validate,Y_validate],(6,6,1)]

In [ ]:
x_validate.shape

# DEFINE ALL POSSIBLE TEST SCENARIOS

In [ ]:
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier

TREE_D = [4,6,8,10]
TREE_F = [0.1,0.2,0.3,0.4,0.5,0.6,0.7]


KERNEL = ['linear','gaussian']
KERNEL_PRUNE = [True,False]
KERNEL_PRUNE = [True]
KERNEL_PRUNE_C = [10]
KERNEL_PRUNE_N = [0.1]

n_estimators = 2

estimators=[]

#for rf_ratio in [0.,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
for rf_ratio in [0.]:
    for t in TREE_D:
        for f in TREE_F:
            tmp = []
            tmp.append('RF')
            tmp.append(t)
            tmp.append(f)
            tmp.append([RandomForestClassifier(max_depth=t,max_features=f,random_state=i,n_jobs=NJOBS,n_estimators=30) for i in range(n_estimators)])
            estimators.append(tmp)  

            for k in KERNEL:
                for pr in KERNEL_PRUNE:
                    if pr:
                        for c in KERNEL_PRUNE_C:
                            for n in KERNEL_PRUNE_N:
                                tmp = []
                                tmp.append(rf_ratio)
                                tmp.append('KF')
                                tmp.append(t)
                                tmp.append(f)
                                tmp.append(k)
                                tmp.append("Pruned")
                                tmp.append(c)
                                tmp.append(n)
                                tmp.append([co2f.CO2_forest(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=1000,kernel=k,max_deth=t,n_jobs=NJOBS,sample_ratio=1.0, feature_ratio = f,n_estimators=30,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.,criteria='gain',reinforced = True, prune_level = n,reC=c,univariate_ratio = rf_ratio) for i in range(int(n_estimators))])                            
                                #tmp[len(tmp) - 1] += [RandomForestClassifier(max_depth=t,max_features=f,random_state=i,n_jobs=NJOBS,n_estimators=10) for i in range(int(n_estimators * rf_ratio))]

                                estimators.append(tmp) 
                    else:
                        tmp = []
                        tmp.append(rf_ratio)

                        tmp.append('KF')
                        tmp.append(t)
                        tmp.append(f)
                        tmp.append(k)
                        tmp.append("Not pruned") 
                        tmp.append([co2f.CO2_forest(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=1000,kernel=k,max_deth=t,n_jobs=NJOBS,sample_ratio=1.0, feature_ratio = f,n_estimators=30,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.,criteria='gain',univariate_ratio = rf_ratio) for i in range(int(n_estimators))])                            
                        #tmp[len(tmp) - 1] += [RandomForestClassifier(max_depth=t,max_features=f,random_state=i,n_jobs=NJOBS,n_estimators=10) for i in range(int(n_estimators * rf_ratio))]

                        estimators.append(tmp)  
                    
WIN = [True,False]

DS_WIN_SZ = {
    'MNIST':[10],
    'USPS':[6],
    'letter':[4],
    'CIFAR':[10],
    'satimage':[6]
}


# Do tests

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import time

def LOG(str_):
    with open("log_2023_diffrent_forests3.txt", "a") as f:
        if str_ != '\n':
            f.write(str(str_) + ";")
        else:    
            f.write(str_)
            
for TRAIN_SIZE in [5000]:
    for TEST_SIZE in [1000]:
        for d in data:
            x_train = data[d][0][0]
            Y_train = data[d][0][1]
            x_validate = data[d][1][0]
            Y_validate = data[d][1][1]
            if SHAKE:
                tr_idxs = numpy.random.permutation(x_train.shape[0])[:TRAIN_SIZE]         
                tst_idxs = numpy.random.permutation(x_validate.shape[0])[:TEST_SIZE]         
                x_train = x_train[tr_idxs]
                Y_train = Y_train[tr_idxs]
                x_validate = x_validate[tst_idxs]
                Y_validate = Y_validate[tst_idxs]

            if data[d][2][2] > 1:
                x_train = x_train.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))
                x_validate = x_validate.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))
            else:
                x_train = x_train.reshape((-1,data[d][2][0],data[d][2][1]))
                x_validate = x_validate.reshape((-1,data[d][2][0],data[d][2][1]))

            for w in WIN:
                for wss in DS_WIN_SZ[d]:
                    for e_idx,e in enumerate(estimators):
                        est = e[len(e) - 1]    
                        model = CascadeForestClassifier(max_layers=3)
                        model.set_estimator(est)   
                        start_time = time.time()
                        if w and data[d][2][0] < wss:
                            model.set_win([wss,-1,-1],[2,1,1],[data[d][2][0],1,1],[data[d][2][1],1,1],[0,0,0],[2,0,0],[False,True,True])
                            model.fit(x_train, Y_train)#
                            pred = model.predict(x_validate)
                        else: 
                            model.fit(x_train.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]), Y_train)#
                            pred = model.predict(x_validate.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))

                        for i in range(len(e) - 1):
                            LOG(e[i])                    
                        LOG(d)
                        LOG(time.time() - start_time)
                        LOG(TRAIN_SIZE)
                        LOG(TEST_SIZE)
                        print (d,w,wss,e[:len(e)-1])


                        LOG(precision_score(Y_validate, pred,average='macro'))
                        LOG(recall_score(Y_validate, pred,average='macro'))                        
                        LOG(accuracy_score(Y_validate, pred))     

                        LOG('\n')
